In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, ToolMessage
load_dotenv()

tool = TavilySearchResults(max_results=2)
# tool.invoke("who is anmol goel")

In [24]:
from modules.chatbot.chatbot import chatbot, state

llm = chatbot(model_name = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free" , model_provider = "together")

llm.set_config({
    "configurable" : {"thread_id" : "2"}
})


llm.set_system_prompt("You are a helpful AI agent with the task of helping the user in any way possible. Dont give any answer larger than 100 words, but also dont make them too short either.Use tools only when absolutely necessary")


In [33]:
llm.model = llm.model.bind_tools([tool])

llm.invoke("sarch the web for the trending movies")

''

In [31]:
from langgraph.graph import END,START

class BasicToolNode ():
    def __init__(self , tools):
        self.tool_by_name = {tool.name : tool for tool in tools}

    def __call__(self , input):
        if messages := input.get("mesasges" , []):
            message = messages[-1]
        else:
            return Exception("No message passed")

        output = []

        for tool_call in message.tool_calls:
            tool_output = self.tool_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )

            output.append(
                ToolMessage(
                    content=tool_output,
                    name = tool_call['name'],
                    id = tool_call["id"]
                )    
            )

        return {messages: output}
    

def route_tools(state : state):
    ai_message = state["messages"][-1]

    if hasattr(ai_message,"tool_calls") and len(ai_message["tool_calls"]) > 0:
        return "tool"
    
    return "END"
    


tool_node = BasicToolNode([tool])

llm.graphBuilder.add_node("tool" , tool_node)
llm.graphBuilder.add_edge("tool" , "llm")
llm.graphBuilder.add_conditional_edges("llm" , route_tools , {"END" : END , "tool" : "tool"})



Adding a node to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.
Adding an edge to a graph that has already been compiled. This will not be reflected in the compiled graph.
